In [132]:
#%% Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bokeh # Bokeh is an interactive visualization library for modern web browsers. 
import folium # leaflet map visualization 

## What does thi code do:
## get the data
## visualize chart with matplotlib
## visualize leaflet map with folium
## create interactive map with bokeh on web browser

## Task 1: getting datasets

In [133]:
def get_data(data_name):
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{data_name}_global.csv'\
            .format(data_name = data_name)
    
    # Get data to dataframe
    data = pd.read_csv(url)
    
    #reshape data from wide to long data with melt function in pandas
    data = pd.melt(data,id_vars=data.columns[:4], var_name='Day', value_name=data_name)
    data[data_name].astype('int') # convert data to numeric
    return data

In [134]:
#%% Get data
confirmed_df = get_data('confirmed') # time_series_covid19_confirmed_global.csv

deaths_df = get_data('deaths') # time_series_covid19_deaths_global.csv

recovered_df = get_data('recovered') # time_series_covid19_recovered_global.csv

In [135]:
confirmed_df.head()
deaths_df.head()
recovered_df.head()

,Province/State,Country/Region,Lat,Long,Day,recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


In [136]:
## merge Confirmed, death, and recovered into one dataframe
keys = ['Province/State','Country/Region', 'Lat','Long','Day']

all_data = confirmed_df
all_data = all_data.merge(deaths_df, on = keys, how = 'inner' )
all_data = all_data.merge(recovered_df, on = keys, how = 'inner' )
all_data['Day'] = pd.to_datetime(all_data['Day'])
all_data['Province/State'] = all_data['Province/State'].fillna('')

all_data['Province/State_Country/Region'] = all_data['Province/State'].fillna('')
all_data['Province/State_Country/Region'] = all_data['Province/State_Country/Region'].apply(lambda x: x if x == '' else x + ' - ')
all_data['Province/State_Country/Region'] = all_data['Province/State_Country/Region'] + all_data['Country/Region']
# delete unused variables.
#del confirmed_df, deaths_df, recovered_df, key



In [137]:
all_data['Province/State'].unique()
all_data['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India', 'Indone

In [138]:
# extract newest data
newest_data =  all_data[all_data['Day']== all_data.Day.max()]

In [139]:
newest_data

,Province/State,Country/Region,Lat,Long,Day,confirmed,deaths,recovered,Province/State_Country/Region
17280,,Afghanistan,33.000000,65.000000,2020-04-03,281,6,10,Afghanistan
17281,,Albania,41.153300,20.168300,2020-04-03,304,17,89,Albania
17282,,Algeria,28.033900,1.659600,2020-04-03,1171,105,62,Algeria
17283,,Andorra,42.506300,1.521800,2020-04-03,439,16,16,Andorra
17284,,Angola,-11.202700,17.873900,2020-04-03,8,2,1,Angola
17285,,Antigua and Barbuda,17.060800,-61.796400,2020-04-03,15,0,0,Antigua and Barbuda
17286,,Argentina,-38.416100,-63.616700,2020-04-03,1265,39,266,Argentina
17287,,Armenia,40.069100,45.038200,2020-04-03,736,7,43,Armenia
17288,Australian Capital Territory,Australia,-35.473500,149.012400,2020-04-03,91,1,18,Australian Capital Territory - Australia
17289,New South Wales,Australia,-33.868800,151.209300,2020-04-03,2389,12,4,New South Wales - Australia


## Task 2: Creat World map

In [140]:
## create folium map with default value
lat, long =10.82302, 106.62965 # this is Ho Chi Minh City
tile = 'cartodbdark_matter'
zoom = 5
my_map =folium.Map(
                location=[lat, long],
                tiles=tile,
                zoom_start=zoom
                )

# adding layer to map
#folium.TileLayer('cartodbpositron').add_to(my_map)

# adding control layer
folium.LayerControl().add_to(my_map)
my_map

In [141]:
# add mouse position
update_day = all_data.Day.max().strftime('%B %d %Y')
prefix = 'Data source: JHU. Update day: {update_day}.<br>Coordinates:'.format(update_day= update_day)
from folium.plugins import MousePosition
formatter = "function(num) {return L.Util.formatNum(num, 3) + '&#176';};"
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='',
    lng_first=True,
    num_digits=20,
    prefix= prefix,
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(my_map)

In [142]:
my_map

In [143]:
from bokeh.plotting import figure
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.models import Legend
from bokeh.embed import file_html
from bokeh.resources import INLINE

# create mini chart with bokeh



In [156]:
## add data to folium map
from folium import IFrame
for i in range(newest_data.shape[0]):
    lat = newest_data.Lat.iloc[i]
    long = newest_data.Long.iloc[i]
    radius = int(3 * newest_data.confirmed.iloc[i])
    
    if newest_data['Province/State'].iloc[i]=='':
        state = newest_data['Country/Region'].iloc[i]
    else:
        state = newest_data['Province/State_Country/Region'].iloc[i]
    confirmed = newest_data['confirmed'].iloc[i]
    recovered = newest_data['recovered'].iloc[i]
    deaths = newest_data['deaths'].iloc[i]
    existing = confirmed - recovered - deaths
    
    # Format toolstip
    tooltip = '''<div background-color="black">
                <p style="text-align: center;"><strong>{state}</strong></p>
                <div style="color: #736E6D; text-align:left;">Confirmed: <span style="color:red"><b> {confirmed}</b> </span></div >
                <div style="color: #736E6D; text-align:left;">Deaths: <span ><b> {deaths}</b> </span></div >
                <div style="color: #736E6D; text-align:left;">Recovered: <span style="color:#37FA02"><b> {recovered}</b> </span></div>
                <div style="color: #736E6D; text-align:left;">Existing: <span style="color:#FA7202"><b>{existing}</b> </span></div>
                </div>
            '''
    tooltip = tooltip.format(state= state.upper(), confirmed= confirmed,
                             recovered= recovered, deaths= deaths,
                             existing= existing)

    folium.Circle(
            location= [lat,long],
            popup=state,
            radius=radius,
            color='crimson',
            fill=True,
            fill_color='crimson',
            weight= 0.2,
            tooltip =tooltip
           ).add_to(my_map)

    

In [157]:
my_map